In [139]:
import re

from time import sleep
from selenium import webdriver
from selenium.webdriver.support.ui import Select

import numpy as np
import pandas as pd
#installation guide for selenium
#https://medium.com/@shanyitan/how-to-install-selenium-and-run-it-successfully-via-jupyter-lab-c3f50d22a0d4

In [140]:
#gathering information function
def gather(browser):
    lst=[]
    try:
        x=1
        while True: 
            x_pather = "//*[@id='myBody']/table/tbody/tr/td/table/tbody/tr[5]/td/form/table/tbody/tr[3]/td/a["+str(x)+"]"
            main = browser.find_element_by_xpath(x_pather)
            x+=1
            lst.append(main)
    except:
        lst.pop()
    return lst

### Scraping the data with selenium
*TODO we need to make it run by page

*TODO testing with other maslulim

In [141]:
browser = webdriver.Chrome()
browser.get("https://shnaton.huji.ac.il/")



select = Select(browser.find_element_by_id('faculty'))
select.select_by_value('07') #Pass value as string
sleep(1)

select = Select(browser.find_element_by_id('hugim'))
select.select_by_value('0824')
sleep(1)

select = Select(browser.find_element_by_id('maslul'))
select.select_by_value('7411')
sleep(1)

select = Select(browser.find_element_by_id('toar'))
select.select_by_value('1')
sleep(1)

select = Select(browser.find_element_by_id('shana'))
select.select_by_value('0')
sleep(1)

browser.find_element_by_css_selector("input[type='radio'][id='full']").click()
sleep(0.2)

browser.find_element_by_css_selector("input[type='button'][value=' חפש ']").click()

#collecting information
up = browser.find_element_by_xpath('//*[@id="myBody"]/table/tbody/tr/td/table/tbody/tr[5]/td/form/table/tbody/tr[3]/td/table[3]/tbody/tr/td[3]/a[2]')
try:
    while True:#information gathering function here
        browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        sleep(1)
        lst=gather(browser)
        up.click()
except:
    print(lst)
    browser.close



[<selenium.webdriver.remote.webelement.WebElement (session="cde6d9fd18bd0f39f84eae3991cb1e27", element="353b72cb-2dd1-4136-be14-283c5338a543")>, <selenium.webdriver.remote.webelement.WebElement (session="cde6d9fd18bd0f39f84eae3991cb1e27", element="26986103-0b7a-4cd6-a654-a746103434ae")>, <selenium.webdriver.remote.webelement.WebElement (session="cde6d9fd18bd0f39f84eae3991cb1e27", element="26541e05-2cb8-4ee4-b4f4-38d16191e01b")>, <selenium.webdriver.remote.webelement.WebElement (session="cde6d9fd18bd0f39f84eae3991cb1e27", element="7df1d98f-91ed-47d9-8d4e-4beb6681c160")>, <selenium.webdriver.remote.webelement.WebElement (session="cde6d9fd18bd0f39f84eae3991cb1e27", element="586cc9d5-1a89-4f8f-905c-4cb09c67fce4")>, <selenium.webdriver.remote.webelement.WebElement (session="cde6d9fd18bd0f39f84eae3991cb1e27", element="4bc68c5a-6fd2-44c7-9b55-eaefaa260356")>, <selenium.webdriver.remote.webelement.WebElement (session="cde6d9fd18bd0f39f84eae3991cb1e27", element="b9d43e3a-5d9d-4997-8cc2-82d4aaca

In [142]:
scraped_data = [i.text for i in lst]

### Gathering the data from the scrpaed text
gathering the data to dictionaries with regex.
we might have some weak points here.
#TODO test with other maslulim.

#### infrastructure 
database = {id:["semester": א\ב\שנתי, "day": א\ב\ג\ד\ה\ו, "hours": "hh:mm-hh:mm",....,..]}

from outside to inside:
big dictionary keys are for course_id, the value is a list which every item in the list is a different hour of the class.

In [143]:
def data_gatherer(scraped_data):
    """ Return (dictionary that contains all the courses per page, canceled courses)"""
    database = dict()
    canceled_courses = []
    for course in scraped_data:
        course_details = []
        info_by_day = dict()


        hours = (re.findall(r"([0-5][0-9]:[0-5][0-9]-[0-5][0-9]:[0-5][0-9])", course))
        course_id = (re.findall(r"\|\s{3}(\d{5})", course))
        day = (re.findall(r"\n(יום [אבגדהו])", course))
        semester = (re.findall(r"\n(שנתי|סמסטר [אבגדהו])", course))

        #checking for canceled courses
        if len(course_id) == 0 or len(hours) == 0:
            if len(hours) == 0:
                canceled_courses.append(course_id)
            pass

        else:
            for item in range(len(hours)):
                info_by_day["semester"] = semester[item] 
                info_by_day["day"] = day[item]
                info_by_day["hours"] = hours[item]
                course_details.append(info_by_day)

            database[course_id[0]] = course_details

    canceled_courses = [x for x in canceled_courses if x != []]
    return database, canceled_courses
data_gatherer(scraped_data)

({'57676': [{'semester': 'סמסטר ב', 'day': 'יום ד', 'hours': '10:15-08:30'},
   {'semester': 'סמסטר ב', 'day': 'יום ד', 'hours': '10:15-08:30'}],
  '57257': [{'semester': 'סמסטר ב', 'day': 'יום ב', 'hours': '12:15-08:30'}],
  '57255': [{'semester': 'סמסטר א', 'day': 'יום ה', 'hours': '10:15-08:30'},
   {'semester': 'סמסטר א', 'day': 'יום ה', 'hours': '10:15-08:30'}],
  '57643': [{'semester': 'שנתי', 'day': 'יום ה', 'hours': '14:15-12:30'}],
  '57639': [{'semester': 'סמסטר ב', 'day': 'יום ד', 'hours': '18:15-16:30'},
   {'semester': 'סמסטר ב', 'day': 'יום ד', 'hours': '18:15-16:30'}],
  '57736': [{'semester': 'שנתי', 'day': 'יום ה', 'hours': '10:15-08:30'}],
  '57152': [{'semester': 'סמסטר א', 'day': 'יום ד', 'hours': '16:15-14:30'},
   {'semester': 'סמסטר א', 'day': 'יום ד', 'hours': '16:15-14:30'}],
  '57811': [{'semester': 'סמסטר א', 'day': 'יום ה', 'hours': '16:15-14:30'},
   {'semester': 'סמסטר א', 'day': 'יום ה', 'hours': '16:15-14:30'}],
  '57200': [{'semester': 'שנתי', 'day': 'י

In [144]:
#helper for regex
# counter = 0
# for i in scraped_data:
#     print(counter)
#     print(i,"\n"+"\n"+"*******end of object********"+"\n"+"\n")
#     counter += 1